In [1]:
import requests
import pdfplumber
import re
import pandas as pd

In [2]:
def download_file(url):
    local_filename = url.split('/')[-1]
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    return local_filename

In [3]:
ap_url = 'https://www.tabs3.com/support/sample/apreports.pdf'

In [4]:
ap = download_file(ap_url)

In [9]:
with pdfplumber.open(ap) as pdf:
    pages = pdf.pages[16]
    text = pages.extract_text()

In [10]:
print(text)

AccountsPayableReportPack
Invoice by Vendor List
Date: 08/26/2024 Invoice by Vendor List Page: 1
Jensen, Martin & Anderson, P.C.
Inv Due Inv Disc Disc Net
Voucher # Invoice # Date Date Amount Amount Date Amount 1099 Ref # Description Codes
200 United Parcel Service
200.01MLJ 3243387 032224 062224 75.00 P 75.00 1 Overnight document delivery * 1
3243387 032324 062324 75.00 P 75.00 3 Postage * 1
3243387 072224 072224 75.00 P 75.00 2 Overnight document delivery * 1
Vendor Totals 225.00 0.00 225.00
202 Software Technology, LLC
312205 081624 082124 490.00 P 490.00 1 Software Maintenance Agreement Tabs3 * 1
312206 081624 082124 239.00 P 239.00 2 Software Maintenance Agreement Tabs3 General Ledger * 1
312207 081624 082124 239.00 P 239.00 3 Software Maintenance Agreement Tabs3 Accounts Payable * 1
312208 081624 082124 239.00 P 239.00 4 Software Maintenance Agreement Tabs3 Trust Accounting * 1
312209 081624 082124 800.00 P 800.00 5 Software Maintenance Agreement PracticeMaster (9 Users) * 1
3122

In [11]:
new_vend_re = re.compile(r'^\d{3} [A-Z].*')

In [13]:
for line in text.split('\n'):
    if new_vend_re.match(line):
        print(line)

200 United Parcel Service
202 Software Technology, LLC
203 Clerk of the County Court
204 Clerk of the District Court
225 Jackson/Wylinda
240 NELCO
250 D & B Real Estate Management Company
300 Thomson Reuters Payment Center
325 Professional Messenger Services
400 Sprint Local & Long Distance
700 Clean All Janitorial Services


In [16]:
for line in text.split('\n'):
    if new_vend_re.match(line):
        vend_num, *vend_name = line.split(' ', 1)   
        print(vend_num, vend_name)

200 ['United Parcel Service']
202 ['Software Technology, LLC']
203 ['Clerk of the County Court']
204 ['Clerk of the District Court']
225 ['Jackson/Wylinda']
240 ['NELCO']
250 ['D & B Real Estate Management Company']
300 ['Thomson Reuters Payment Center']
325 ['Professional Messenger Services']
400 ['Sprint Local & Long Distance']
700 ['Clean All Janitorial Services']


In [17]:
invoice_line_re = re.compile(r'\d{6} \d{6} [\d,]+\.\d{2}')

In [19]:
for line in text.split('\n'):
    if invoice_line_re.search(line):
        print(line)

200.01MLJ 3243387 032224 062224 75.00 P 75.00 1 Overnight document delivery * 1
3243387 032324 062324 75.00 P 75.00 3 Postage * 1
3243387 072224 072224 75.00 P 75.00 2 Overnight document delivery * 1
312205 081624 082124 490.00 P 490.00 1 Software Maintenance Agreement Tabs3 * 1
312206 081624 082124 239.00 P 239.00 2 Software Maintenance Agreement Tabs3 General Ledger * 1
312207 081624 082124 239.00 P 239.00 3 Software Maintenance Agreement Tabs3 Accounts Payable * 1
312208 081624 082124 239.00 P 239.00 4 Software Maintenance Agreement Tabs3 Trust Accounting * 1
312209 081624 082124 800.00 P 800.00 5 Software Maintenance Agreement PracticeMaster (9 Users) * 1
312210 081624 082124 105.00 P 105.00 6 Software Maintenance Agreement Tabs3 Taskbill * 1
320559 082124 082624 205.00 205.00 7 Purchase Tabs3 Device Interface Program 1
200.01PAM 042224 042224 225.00 P 225.00 1 Documentation Fee, Filing Fee, and Sheriff Service Fee * 1
032324 032324 75.00 P 75.00 2 Filing fee * 1
051624 051624 75.0

In [22]:
invoice_line_re = re.compile(r'(\d{6}) (\d{6}) ([\d,]+\.\d{2})')   #grouping


In [27]:
for line in text.split('\n'):
    line = invoice_line_re.search(line)
    if line:
        print(line.group(1), line.group(2), line.group(3))

032224 062224 75.00
032324 062324 75.00
072224 072224 75.00
081624 082124 490.00
081624 082124 239.00
081624 082124 239.00
081624 082124 239.00
081624 082124 800.00
081624 082124 105.00
082124 082624 205.00
042224 042224 225.00
032324 032324 75.00
051624 051624 75.00
081624 081624 150.00
122023 122023 750.00
072224 072224 1,500.00
072224 082224 129.85
071824 071824 7,500.00
072224 072224 7,500.00
080924 080924 10.00
042124 042124 4.00
052024 052024 75.00
062124 062124 40.00
072524 072524 4.00
072624 072624 50.00
072624 072624 50.00
072624 072624 50.00
072624 072624 50.00
061724 061724 20.00
062024 062024 25.00
082124 082124 47.50
082124 082124 35.00
081624 082124 68.96
071824 072124 750.00
072224 082124 750.00


In [74]:
# furthur data
invoice_line_re = re.compile(r'(\d{6}) (\d{6}) ([\d,]+\.\d{2}) ([\sP]*)([\d,]+\.\d{2}) ([YN ]*\d) (.*?) ([*\s\d])')   #grouping

In [76]:
for line in text.split('\n'):
    line = invoice_line_re.search(line)
    if line:
        print(line.group(1), line.group(2), line.group(3), line.group(4), line.group(5), line.group(6), line.group(7), line.group(8))

032224 062224 75.00 P  75.00 1 Overnight document delivery *
032324 062324 75.00 P  75.00 3 Postage *
072224 072224 75.00 P  75.00 2 Overnight document delivery *
081624 082124 490.00 P  490.00 1 Software Maintenance Agreement Tabs3 *
081624 082124 239.00 P  239.00 2 Software Maintenance Agreement Tabs3 General Ledger *
081624 082124 239.00 P  239.00 3 Software Maintenance Agreement Tabs3 Accounts Payable *
081624 082124 239.00 P  239.00 4 Software Maintenance Agreement Tabs3 Trust Accounting *
081624 082124 800.00 P  800.00 5 Software Maintenance Agreement PracticeMaster (9 Users) *
081624 082124 105.00 P  105.00 6 Software Maintenance Agreement Tabs3 Taskbill *
082124 082624 205.00  205.00 7 Purchase Tabs3 Device Interface Program 1
042224 042224 225.00 P  225.00 1 Documentation Fee, Filing Fee, and Sheriff Service Fee *
032324 032324 75.00 P  75.00 2 Filing fee *
051624 051624 75.00 P  75.00 3 Filing fee *
081624 081624 150.00  150.00 1 Petition for Dissolution of Marriage Filing Fe

In [59]:
from collections import namedtuple

In [88]:
# INV = namedtuple('inv', 'vend_num vend_name voucher_num inv_num inv_date due_date disc_amt disc_date net_amount desc')
INV = namedtuple('inv', 'vend_num vend_name inv_date due_date inv_amt net_amt description')

In [102]:
line_items = []

for line in text.split('\n'):
    if new_vend_re.match(line):
        vend_num, *vend_name = line.split(' ', 1)
    
    line = invoice_line_re.search(line)
    if line:
        inv_date = line.group(1)
        due_date = line.group(2)
        inv_amt = line.group(3).replace(',', '')
        net_amt = line.group(5).replace(',', '')
        description = line.group(7).replace(',', '')
        line_items.append(INV(vend_num, vend_name, inv_date, due_date,inv_amt, net_amt, description))

In [103]:
line_items[3]

inv(vend_num='202', vend_name=['Software Technology, LLC'], inv_date='081624', due_date='082124', inv_amt='490.00', net_amt='490.00', description='Software Maintenance Agreement Tabs3')

In [104]:
df = pd.DataFrame(line_items)

In [105]:
df.head()

,vend_num,vend_name,inv_date,due_date,inv_amt,net_amt,description
0,200,[United Parcel Service],032224,062224,75.00,75.00,Overnight document delivery
1,200,[United Parcel Service],032324,062324,75.00,75.00,Postage
2,200,[United Parcel Service],072224,072224,75.00,75.00,Overnight document delivery
3,202,"[Software Technology, LLC]",081624,082124,490.00,490.00,Software Maintenance Agreement Tabs3
4,202,"[Software Technology, LLC]",081624,082124,239.00,239.00,Software Maintenance Agreement Tabs3 General L...


In [106]:
df['inv_date'] = pd.to_datetime(df['inv_date'])
df['due_date'] = pd.to_datetime(df['due_date'])

C:\Users\Vishwajeet\AppData\Local\Temp\ipykernel_22920\4245299104.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['inv_date'] = pd.to_datetime(df['inv_date'])
C:\Users\Vishwajeet\AppData\Local\Temp\ipykernel_22920\4245299104.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['due_date'] = pd.to_datetime(df['due_date'])


In [107]:
df.head()

,vend_num,vend_name,inv_date,due_date,inv_amt,net_amt,description
0,200,[United Parcel Service],2024-03-22,2024-06-22,75.00,75.00,Overnight document delivery
1,200,[United Parcel Service],2024-03-23,2024-06-23,75.00,75.00,Postage
2,200,[United Parcel Service],2024-07-22,2024-07-22,75.00,75.00,Overnight document delivery
3,202,"[Software Technology, LLC]",2024-08-16,2024-08-21,490.00,490.00,Software Maintenance Agreement Tabs3
4,202,"[Software Technology, LLC]",2024-08-16,2024-08-21,239.00,239.00,Software Maintenance Agreement Tabs3 General L...


In [108]:
df['inv_amt'] = df['inv_amt'].map(lambda x: float(x.replace(',', '')))
df['net_amt'] = df['net_amt'].map(lambda x: float(x.replace(',', '')))

In [110]:
df.to_csv('invoices.csv')